- 다음 영화 정보를 수집한다.
    - https://movie.daum.net/premovie/released
    

In [46]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os

from urllib.request import urlopen

# 출력화면 초기화 하는 모듈
from IPython.display import clear_output

### 요청하는 함수

In [3]:
def getSource(site):
    # 헤더 정보
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36',   
    }
    
    # 요청한다.
    response = requests.get(site, headers = header_info)
#     print(response.text)

    # bs4 객체 생성
    soup = bs4.BeautifulSoup(response.text,'lxml')
    
    return soup

### 한 페이지의 데이터를 수집해 저장하는 함수

In [4]:
def getData(soup):
    # 전체를 가지고 있는 태그를 가지고 온다.
    a1 = soup.select_one('#mArticle > ul.list_movie')

    # 영화 데이터 가져오기
    a2 = a1.select('li')
    
    # 데이터를 담을 딕셔너리 생성
    data_dict ={
        '영화제목' :[],
        '평점' : [],
        '개봉날짜': [],
        '예매율' : []
    }
    
    # 영화의 수만큼 반복한다.
    for a3 in a2:
        a4 = a3.select_one('div.wrap_movie > div > a')
        data1 = a4.text.strip()
        
        # 평점을 가져온다.
        a5 = a3.select('div.wrap_movie > span.info_grade > a > span.wrap_grade.grade_netizen > span.num_grade')
        b1 = int(a5[0].text)
        b2 = int(a5[0].text)
        data2 = f'{b1}.{b2}'
        
        # 개봉날짜와 예매율
        a6 = a3.select_one('div.wrap_movie > span.info_state')
        
        if len(a6.select('span')) == 0:
            data31 = a6.text.strip()
            
        elif len(a6.select('span')) == 1:
            b3 = a6.text.split('・')
            
        data3 = b3[0].strip()
        data4 = b3[1].strip()

        # 개봉 날짜에서 '개봉일' 문자열 제거
        data3 = data3.replace('개봉','').strip()
        data3 = data3.replace('재','').strip()
        data4 = data4.replace('예매율','').strip()
        data4 = data4.replace('%','').strip()
        
        # 딕셔너리에 데이터담기
        data_dict['영화제목'].append(data1)
        data_dict['평점'].append(data2)
        data_dict['개봉날짜'].append(data3)
        data_dict['예매율'].append(data4)
        
        
    # 데이터 프레임 생성
    df1 = pd.DataFrame(data_dict)
    print(df1)
    
            
            
#     # 데이터 프레임 저장
#     if os.path.exists('Daum_moive.csv') == False:
#         df1.to_csv('Daum_moive.csv', encoding='utf-8-sig', index=False)
#     else:
#         df1.to_csv('Daum_moive.csv', encoding='utf-8-sig', index=False, header=False, mode='a')
    
    
#         print(data1)
#         print(data2)
#         print(data3)
#         print(data4)
#         print('-' * 20)

In [5]:
site = 'https://movie.daum.net/premovie/released?reservationOnly=N&sort=reservation&page=3'
soup = getSource(site)
getData(soup)

         영화제목     평점      개봉날짜  예매율
0  마이 미씽 발렌타인    8.8  21.01.14  0.0
1     아이 엠 우먼    8.8  21.01.14  0.0
2           걸    8.8  21.01.07  0.0
3     미드나잇 버스  10.10  21.02.04  0.0
4    천사는 바이러스    8.8  21.02.04  0.0


### 다음페이지 존재 여부 확인

In [6]:
def getNextPage(soup):
    pages = soup.select_one('#mArticle > div.paging_movie > span ')
    b = pages.get_text()
    b = b.replace('현재페이지','')
    total_pages= b.split("\n")[1:-1]
    
    for page in total_pages:
        return page
    
#     # next 버튼 태그를 가져온다.
#     for i in range(0,len(total_pages)+1):
#         a=f'https://movie.daum.net/premovie/released?reservationOnly=N&sort=reservation&page={i}'
#         getSource(a)
#         soup = getSource(a)
#         return soup

In [7]:
getNextPage(soup)

'1'

In [7]:
# def getNextPage(soup,i):
#     pages = soup.select('#mArticle > div.paging_movie > span > a')
#     if pages[i].attrs['href'] 
#     return pages[i].attrs['href']

# #     for page in pages:
# #         # 만약 가져온 것이 있다면
# #         if page != None:
# #             # a 태그의 href 속성 값을 가져온다.
# #             href = page.attrs['href']
# #             return href
# #         else:
# #             return None

In [8]:
# site = 'https://movie.daum.net/premovie/released?reservationOnly=N&sort=reservation&page=2'
# soup = getSource(site)
# getNextPage(soup,1)

### 결과확인

In [9]:
site = 'https://movie.daum.net/premovie/released?reservationOnly=N&sort=reservation&page='
page = '1'

while True:
    # 딜레이
    time.sleep(2)
    
    # 기존 출력된거 정리
    clear_output(wait=True)
    
    # 페이지 요청
    soup = getSource(site+page)
    
    # 데이터 수집 및 저장
    getData(soup)
    
    #  다음 페이지 이동
    page = getNextPage(soup)
    
    # 다음페이지가 없다면 중단
    if page == None:
        print('수집완료')
        break

                 영화제목     평점      개봉날짜   예매율
0   극장판 귀멸의 칼날: 무한열차편    5.5  21.01.27  31.5
1                  소울    8.8  21.01.20  21.3
2             어니스트 씨프    6.6  21.02.03   6.2
3              해피 투게더    8.8  21.02.04   3.1
4                 페어웰    8.8  21.02.04   2.1
5           서바이브 더 나잇    5.5  21.02.03   1.8
6               칙 파이트  10.10  21.02.04   1.6
7             스트레스 제로  10.10  21.02.03   1.4
8                북스마트    6.6  21.01.27   1.2
9   세상의 모든 디저트: 러브 사라    0.0  21.02.04   1.2
10               스푸트닉    4.4  21.02.04   1.1
11                세자매    8.8  21.01.27   0.9
12     나는 나를 해고하지 않는다    9.9  21.01.28   0.8
13               사일런싱    6.6  21.01.28   0.8
14                 캐롤    7.7  21.01.27   0.7
15   명탐정 코난: 진홍의 수학여행    7.7  21.01.27   0.6
16       게임의 법칙: 인간사냥    3.3  21.01.28   0.5
17               화양연화    8.8  20.12.24   0.4
18       이 안에 외계인이 있다  10.10  21.02.03   0.2
19    관계의 가나다에 있는 우리는  10.10  21.01.28   0.1


KeyboardInterrupt: 

## 강사님 풀이

### 요청하는 함수

In [26]:
def getSource(site):
    # 헤더 정보
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36',   
    }
    
    # 요청한다.
    response = requests.get(site, headers = header_info)
#     print(response.text)

    # bs4 객체 생성
    soup = bs4.BeautifulSoup(response.text,'lxml')
    
    return soup

### 한 페이지의 데이터를 수집해 저장하는 함수

In [54]:
def getData(soup):
    # 전체를 가지고 있는 태그를 가지고 온다.
    a1 = soup.select_one('#mArticle > ul.list_movie')

    # 영화 데이터 가져오기
    a2 = a1.select('li')
    
    # 데이터를 담을 딕셔너리 생성
    data_dict ={
        '영화제목' :[],
        '평점' : [],
        '개봉날짜': [],
        '예매율' : []
    }
    
    # 영화의 수만큼 반복한다.
    for a3 in a2:
        a4 = a3.select_one('div.wrap_movie > div > a')
        data1 = a4.text.strip()
        
        # 평점을 가져온다.
        a5 = a3.select('div.wrap_movie > span.info_grade > a > span.wrap_grade.grade_netizen > span.num_grade')
        b1 = int(a5[0].text)
        b2 = int(a5[0].text)
        data2 = f'{b1}.{b2}'
        
        # 개봉날짜와 예매율
        a6 = a3.select_one('div.wrap_movie > span.info_state')
        b3 = a6.text.split('・')
        data3 = b3[0].strip()
        
        if len(b3) == 2 :    
            data4 = b3[1].strip()
        else :
            data4 = np.nan
    
        # 개봉 날짜에서 '개봉일' 문자열 제거
        data3 = data3.replace('개봉','').strip()
        data3 = data3.replace('재','').strip()
        if len(b3) == 2:
            data4 = data4.replace('예매율','').strip()
            data4 = data4.replace('%','').strip()
        
        # 딕셔너리에 데이터담기
        data_dict['영화제목'].append(data1)
        data_dict['평점'].append(data2)
        data_dict['개봉날짜'].append(data3)
        data_dict['예매율'].append(data4)
        
        # 영화 포스터 가져오기
        a7 = a3.select_one('div.info_movie > span > img')
        # src 속성값을 가져온다.
        src_attr = a7.attrs['src']
        print(src_attr)
        
        if len(src_attr)>0:
            # 포스터 이미지가 존재할 경우
            # 이미지 데이터를 읽어온다.
            with urlopen('http:' + src_attr) as f1:
                img_data = f1.read()
                
            # 영화 제목의 특수문자를 제거한다.
            filename = data1.replace(':','-')
            filename = filename.replace('/',' ')
            filename = filename.replace(';','')
            
            
            # 저장
            with open(f'poster/{filename}.jpg','wb') as f2:
                f2.write(img_data)
        
        
        
        
        
#     # 데이터 프레임 생성
#     df1 = pd.DataFrame(data_dict)
#     display(df1)
    
    
            
            
#     # 데이터 프레임 저장
#     if os.path.exists('Daum_moive.csv') == False:
#         df1.to_csv('Daum_moive.csv', encoding='utf-8-sig', index=False)
#     else:
#         df1.to_csv('Daum_moive.csv', encoding='utf-8-sig', index=False, header=False, mode='a')
    
    
#         print(data1)
#         print(data2)
#         print(data3)
#         print(data4)
#         print('-' * 20)

In [55]:
site = 'https://movie.daum.net/premovie/scheduled?reservationOnly=N&sort=reservation&page=2'
soup = getSource(site)
getData(soup)

//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/b6a55c2b29c5dd00700e6226283f79e72292f882
//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/6006f9662caa592e42cb7dbda628260ab5078a92
//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/1b55b9499efe917dff37b5b122232f4fded7fcde
//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/a4844e99f4278db0f745db46a4858b6669ff9cb2
//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/d80c63d8d759d43efbcfc0aa62dfd6419326758c
//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/5709f9cfabc9537fc26610e98a6197556a315e31
//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/f16ee067963316310d61dd068e19d2ccb4fad24f

//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/489848c875d8c1ab27edeb8d1b2ca63d0034f935
//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/7924c44d525d793e68103a41de3f29aa4701cae2


### 다음페이지 존재 여부 확인

In [56]:
def getNext(soup):
    # 페이지 번호중 a 태그를 가져온다.
    a_tag = soup.select('#mArticle > div.paging_movie > span > a')
    
    # 페이지 번호중 em 태그를 가져온다.
    em_tag = soup.select_one('#mArticle > div.paging_movie > span > em')

    # a 태그 중 제일 마지막 태그의 숫자 값을 가져온다.
    a1 = a_tag[-1].text.strip()
    
    # em 태그의 숫자값을 가져온다.
    a2 = em_tag.text.strip()
    a2 = a2.replace('현재페이지','')
    
    if int(a1) > int(a2):
        return True
    else:
        return False

In [57]:
site = 'https://movie.daum.net/premovie/released?reservationOnly=N&sort=reservation&page=3'
soup = getSource(site)
getNext(soup)

False

### 결과확인

In [58]:
page_name = 'released'
page = 1

while True:
    # 주소
    site = f'https://movie.daum.net/premovie/{page_name}?reservationOnly=N&sort=reservation&page={str(page)}'
    
    print(f'{page_name} : {page} 수집중')
    # 딜레이
    time.sleep(2)
    
    # 기존 출력된거 정리
    clear_output(wait=True)
    
    # 페이지 요청
    soup = getSource(site)
    
    # 데이터 수집 및 저장
    getData(soup)
    
    #  다음 페이지 이동
    chk = getNext(soup)
    
    # 다음페이지가 없다면 중단
    if chk == False:
        print('수집완료')
        if page_name =='released':
            page_name = 'scheduled'
            page =1
        else:
              break
    else: 
        page += 1

//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/2b1d3188c817fa8feb1c3f78a2625ec71481c673
//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/e6827930fecc0f415ee58d3d697be22d7fcba989
//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/8f6d909119d250749366527dfaa0e7977e165f22
//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/7efe89a90a6ba8d42acf5613746d0fe8cb03d40e
//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/5c3475daf193ecf66a90278c5dafafc3ee69782d

//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/daba12924aec5f38a76d5143cbe62edacc605e7a
//img1.daumcdn.net/thumb/C236x340/?fname=https://t1.daumcdn.net/movie/ca80929bf7acd77089fc9656bda496cfe4855d08
수집완료


### 최종 데이터 프레임을 생성

In [36]:
df10 = pd.read_csv('Daum_moive.csv')
df10

,영화제목,평점,개봉날짜,예매율
0,극장판 귀멸의 칼날: 무한열차편,5.5,21.01.27,31.6
1,소울,8.8,21.01.20,25.5
2,어니스트 씨프,6.6,21.02.03,5.9
3,해피 투게더,8.8,21.02.04,2.9
4,페어웰,8.8,21.02.04,1.8
...,...,...,...,...
147,꿈꾸는 고양이,10.1,21.06.04,NaN
148,달이 지는 밤,10.1,21.09.01,NaN
149,물방울을 그리는 남자,0.0,21.10.19,NaN
150,007 노 타임 투 다이,7.7,21.10,NaN
